In [1]:
import pandas as pd #imports
import numpy as np
import multiprocessing

from fbprophet import Prophet
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error as mae
from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np
import scipy.stats

In [2]:
def preprocess_download(data, start_date_back, end_date_back): #preprocess data
    data = data.dropna(subset=['subregion1_code'])
    data = data[data.country_code == 'US']

    data = data[['date',
                 'subregion1_code',
                 'total_deceased',
                 'average_temperature', 
                 'minimum_temperature',
                 'maximum_temperature', 
                 'rainfall', 
                 'dew_point', 
                 'relative_humidity',
                 'new_confirmed']]
    
    data['days_back'] = [(datetime.now() - day).days for day in pd.to_datetime(data.date)]
    data = data[data['days_back']<=start_date_back]
    data = data[data['days_back']>=end_date_back]
    data.fillna(0, inplace = True)
    
    res = {}
    for state in data.subregion1_code.unique():
        if state in ['MP', 
                     'AS',
                     'GU',
                     'PR', 
                     'VI']:
            continue
        print('State: ', state)
        tmp_state = data[data.subregion1_code == state]
        tmp = []
        for date in tmp_state.date.unique():
            tmp_state_day = tmp_state[tmp_state.date == date]
            tmp.append({'ds': pd.to_datetime(date), 
                        'total_deceased': sum(tmp_state_day.total_deceased),
                        'average_temperature': np.mean(tmp_state_day.average_temperature),
                        'minimum_temperature': np.mean(tmp_state_day.minimum_temperature),
                        'maximum_temperature': np.mean(tmp_state_day.maximum_temperature),
                        'rainfall': np.mean(tmp_state_day.rainfall),
                        'dew_point': np.mean(tmp_state_day.dew_point),
                        'relative_humidity': np.mean(tmp_state_day.relative_humidity),
                        'y': sum(tmp_state_day.new_confirmed)})
        res[state] = pd.DataFrame(tmp)
    
    return(res)

In [3]:
data = preprocess_download(pd.read_csv("https://storage.googleapis.com/covid19-open-data/v2/main.csv", 
                                       low_memory=False), 
                           62, 
                           2)


State:  AK
State:  AL
State:  AR
State:  AZ
State:  CA
State:  CO
State:  CT
State:  DC
State:  DE
State:  FL
State:  GA
State:  HI
State:  IA
State:  ID
State:  IL
State:  IN
State:  KS
State:  KY
State:  LA
State:  MA
State:  MD
State:  ME
State:  MI
State:  MN
State:  MO
State:  MS
State:  MT
State:  NC
State:  ND
State:  NE
State:  NH
State:  NJ
State:  NM
State:  NV
State:  NY
State:  OH
State:  OK
State:  OR
State:  PA
State:  RI
State:  SC
State:  SD
State:  TN
State:  TX
State:  UT
State:  VA
State:  VT
State:  WA
State:  WI
State:  WV
State:  WY


In [4]:
def run_prophet(data, forecast_dist, state): # Facebook Prophet run on all states
    def mean_confidence_interval(data, confidence=0.95): #create confidence interval not for metric but to track through iteration
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return h
    
    def Prophet_reg(df): #Run Facebook Prophet
        prophet = Prophet(yearly_seasonality=False, daily_seasonality=True)
        for col in df.columns:
            if col not in ['ds','y']:
                prophet.add_regressor(col)
        prophet.fit(df)
        return prophet
    
    def make_future_dataframe(df): #make future DataFrame
        future = prophet.make_future_dataframe(periods=forecast_dist, freq='D')

        for col in df.columns:
            if col not in ['ds','y']:
                future[col] = df[col]
        future.fillna(0, inplace = True)
        return future
    
    df = data[state]
    
    y_train_1, y_test_1 = temporal_train_test_split(df, test_size=forecast_dist)  #train test split
    prophet = Prophet_reg(y_train_1)
    future = make_future_dataframe(y_train_1)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_1):]
    oot_val_1 = (y_test_1['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_2, y_test_2 = temporal_train_test_split(y_train_1, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_2)
    future = make_future_dataframe(y_train_2)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_2):]
    oot_val_2 = (y_test_2['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_3, y_test_3 = temporal_train_test_split(y_train_2, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_3)
    future = make_future_dataframe(y_train_3)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_3):]
    oot_val_3 = (y_test_3['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_4, y_test_4 = temporal_train_test_split(y_train_3, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_4)
    future = make_future_dataframe(y_train_4)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_4):]
    oot_val_4 = (y_test_4['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    y_train_5, y_test_5 = temporal_train_test_split(y_train_4, test_size=forecast_dist)
    prophet = Prophet_reg(y_train_5)
    future = make_future_dataframe(y_train_5)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(y_train_5):]
    oot_val_5 = (y_test_5['y'].iloc[-1:] - fb_preds.values[-1:]).values[0]
    
    ci = mean_confidence_interval([oot_val_1, oot_val_2, oot_val_3, oot_val_4, oot_val_5])
    
    prophet = Prophet_reg(df)
    future = make_future_dataframe(df)
    forecast = prophet.predict(future)
    fb_preds = forecast['yhat'].iloc[len(df):]
    pred_future = fb_preds.values[-1:][0]
    return {'State': state,
            'ci_Prediction_%': np.round(ci/pred_future*100),
            'Prediction_95_ci_l': pred_future-ci,
            'Prediction': pred_future,  
            'Prediction_95_ci_h': pred_future+ci}

In [5]:
num_cores = multiprocessing.cpu_count() # run on AWS in parallel
state_list = tqdm(list([state for state in data.keys()]))
processed_list = Parallel(n_jobs=num_cores)(delayed(run_prophet)(data, 
                                                                 3, 
                                                                 state) for state in state_list)

100%|██████████| 51/51 [00:35<00:00,  1.44it/s]


In [6]:
state_level_predictions = pd.DataFrame(processed_list)
# state_level_predictions.sort_values(['Prediction'], ascending = False)
# state_level_predictions.sort_values(['ci_Prediction_%'], ascending = False)
state_level_predictions.sort_values(['Prediction'], ascending = True)

,State,ci_Prediction_%,Prediction_95_ci_l,Prediction,Prediction_95_ci_h
14,IL,-8.0,-205648.000644,-190818.138070,-175988.275496
31,NJ,-102.0,-294216.364205,-145456.632685,3303.098836
22,MI,-59.0,-138075.690615,-87068.179466,-36060.668317
38,PA,-17.0,-96907.707699,-83153.104748,-69398.501797
6,CT,-68.0,-126105.813382,-75272.510082,-24439.206782
19,MA,-90.0,-136308.366917,-71583.464877,-6858.562838
27,NC,-46.0,-83275.436230,-56921.148959,-30566.861687
42,TN,-41.0,-42630.605671,-30196.760433,-17762.915194
5,CO,-25.0,-25265.545059,-20275.058237,-15284.571416
24,MO,-94.0,-37876.949001,-19477.421748,-1077.894494


In [7]:
state_level_predictions.to_csv('./statepreds.csv')